# Loader for Edward sound lib

This library includes both footsteps and the sound of moving with different material clothes.

In [2]:
CATEGORY = ("footsteps", "clothes_movement") # TODO: Change!
SOURCE_NAME = "edward_v1.1"
ORIGINAL_PATH = "./data/original/" + SOURCE_NAME + "/"
FOOTSTEPS_PATH = ORIGINAL_PATH + "WAV/Footsteps/"
CLOTHES_PATH = ORIGINAL_PATH + "WAV/Clothes_Movement/"
TARGET_PATH = "./data/intermediate/" + SOURCE_NAME + "/"

## Get the filenames to parse

In [3]:
import os

In [4]:
audio_extensions = (".wav", ".mp3", ".flac", ".aac", ".m4a", ".ogg", ".opus")

def load_filenames(path: str) -> list[str]:
    "Load a list of audio filenames in the directory."
    return [filename for filename in os.listdir(path)
            if filename.endswith(audio_extensions)]

In [6]:
clothes_filenames = load_filenames(CLOTHES_PATH)
clothes_filenames[:5]

['589_Foley_Clothes_Movement_Medium_Normal_Walk_Close.wav',
 '563_Foley_Clothes_Movement_Jeans_Jump_Land_Special_Close.wav',
 '562_Foley_Clothes_Movement_Jeans_Fast_Walk_Run_Distance.wav',
 '571_Foley_Clothes_Movement_Jeans_Very_Slow_Walk_Creep_Close.wav',
 '573_Foley_Clothes_Movement_Light_Fast_Walk_Run_Close.wav']

In [7]:
extensions = [filename.split(".")[-1] for filename in clothes_filenames]
assert len(extensions) == len(clothes_filenames)
extensions[:5]

['wav', 'wav', 'wav', 'wav', 'wav']

## Parse the files into labels

* TODO: Complete

In [8]:
import string
import stringcase

In [9]:
# We have to lower() first otherwise stringcase.snakecase will prepend multiple underscores
def normalise_label(label: str) -> str:
    return stringcase.capitalcase(stringcase.snakecase(label.lower().strip()))

In [10]:
def get_labels(filename: str) -> list[str]:
    "Parse the filename to get a list of labels."
    # Remove the prefix and extension
    return [filename.split("Clothes_Movement_", maxsplit=1)[1].rsplit(".", maxsplit=1)[0]]

In [12]:
labels = [get_labels(filename) for filename in clothes_filenames]
assert len(labels) == len(clothes_filenames)
labels

[['Medium_Normal_Walk_Close'],
 ['Jeans_Jump_Land_Special_Close'],
 ['Jeans_Fast_Walk_Run_Distance'],
 ['Jeans_Very_Slow_Walk_Creep_Close'],
 ['Light_Fast_Walk_Run_Close'],
 ['Medium_Fast_Walk_Run_Distance'],
 ['Light_Normal_Walk_Distance'],
 ['Jeans_Stop_Small_Special_Close'],
 ['Medium_Very_Slow_Walk_Creep_Close'],
 ['Medium_Normal_Walk_Distance'],
 ['Light_Stop_Small_Special_Distance'],
 ['Medium_Slow_Walk_Distance'],
 ['Light_Fast_Walk_Run_Distance'],
 ['Medium_Jump_Land_Special_Close'],
 ['Jeans_Jump_Land_Special_Distance'],
 ['Light_Slow_Walk_Close'],
 ['Jeans_Stop_Small_Special_Distance'],
 ['Light_Slow_Walk_Distance'],
 ['Light_Stop_Small_Special_Close'],
 ['Jeans_Very_Slow_Walk_Creep_Distance'],
 ['Jeans_Normal_Walk_Distance'],
 ['Medium_Slow_Walk_Close'],
 ['Medium_Stop_Small_Special_Close'],
 ['Light_Jump_Land_Special_Close'],
 ['Medium_Very_Slow_Walk_Creep_Distance'],
 ['Medium_Fast_Walk_Run_Close'],
 ['Medium_Jump_Land_Special_Distance'],
 ['Jeans_Normal_Walk_Close'],
 ['J

## Copy the files to the intermediate folder

In [14]:
import xxhash
import shutil

In [15]:
def get_hash(filename: str) -> str:
    "Get the sha2 hash of the file."
    with open(os.path.normpath(filename), "rb") as f:
        return xxhash.xxh64(f.read()).hexdigest()

In [17]:
hashed_filename = ["{}.{}".format(get_hash(CLOTHES_PATH + filename), extension)
                   for filename, extension in zip(clothes_filenames, extensions)]
assert len(hashed_filename) == len(clothes_filenames)
hashed_filename[:5]

['2fe56997f8de5b02.wav',
 '5e0fc337b16bc512.wav',
 'ad7eb7674ef0c3a6.wav',
 '289b7703cab05b0a.wav',
 '55241b0f36af1bb7.wav']

In [18]:
if not os.path.exists(TARGET_PATH):
    os.makedirs(TARGET_PATH)

In [19]:
# Copy new the files to the target directory
for filename, hash in zip(clothes_filenames, hashed_filename):
    target_file_path = os.path.join(TARGET_PATH, hash)
    if not os.path.exists(target_file_path):
        shutil.copy2(CLOTHES_PATH + filename, target_file_path)

## Export the CSV rows

In [20]:
import csv

In [21]:
with open("version", "r") as f:
    version = int(f.read())
print("Version:", version)

Version: 9


In [24]:
# Create a CSV file with the labels
category_rows = [normalise_label(CATEGORY[1])] * len(labels)
label_rows = [",".join(l) for l in labels]
extra_rows = [""] * len(labels)
source_rows = [SOURCE_NAME] * len(labels)
version_rows = [version] * len(labels)

with open(TARGET_PATH + "metadata.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "category", "label", "extra", "source", "version"])
    writer.writerows(zip(hashed_filename, category_rows, label_rows, extra_rows, source_rows, version_rows))